In [1]:
from boututils.datafile import DataFile
from boutdata.collect import collect
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, sys, pathlib
import platform
import traceback
import xarray as xr
import xbout
from pathlib import Path
import xhermes as xh
import matplotlib.pyplot as plt
import os, sys, pathlib
import numpy as np
import xarray as xr
import xhermes as xh

sys.path.append(os.path.join(r"/users/jlb647/scratch/simulation_program/hermes-3_sim/analysis/sdtools"))
sys.path.append(os.path.join(r"/users/jlb647/scratch/simulation_program/hermes-3_sim/analysis/my_notebooks/notebooks/hermes-3/general_functions"))

from convergence_functions import *

from matplotlib.ticker import LogFormatter

sys.path.append(os.path.join(r"/users/jlb647/scratch/simulation_program/hermes-3_sim/analysis/sdtools"))
sys.path.append(os.path.join(r"/users/jlb647/scratch/simulation_program/hermes-3_sim/analysis/my_notebooks/notebooks/hermes-3/transients"))
sys.path.append(os.path.join(r"/users/jlb647/scratch/simulation_program/hermes-3_sim/analysis/my_notebooks/notebooks/hermes-3/general_functions"))


from plotting_functions import *
from convergence_functions import * 

from hermes3.case_db import *
from hermes3.load import *
from hermes3.named_selections import *
from hermes3.plotting import *
from hermes3.grid_fields import *
from hermes3.accessors import *
from hermes3.utils import *
from hermes3.fluxes import *


# plt.style.use('ggplot')
plt.rcParams.update({'font.size': 10})
linewidth = 3
markersize = 15



# plt.style.use('ggplot')
plt.style.use('default')
plt.rcParams["axes.edgecolor"] = "black"
plt.rcParams["axes.linewidth"] = 1
plt.rcParams['xtick.labelsize'] = 18
plt.rcParams['ytick.labelsize'] = 18
plt.rcParams['axes.grid'] = True
plt.rcParams.update({'font.size': 16})



%load_ext autoreload
%autoreload 2


In [2]:
fl_cd = xh.open('/users/jlb647/scratch/simulation_program/hermes-3_sim/simulation_dir/2024-10_glimmer_working_cases/FL/CD/FL_CD_5.0_w')
fl_sxd = xh.open('/users/jlb647/scratch/simulation_program/hermes-3_sim/simulation_dir/2024-10_glimmer_working_cases/FL/SXD/FL_SXD_w_cdtosxd')

snb_cd = xh.open('/users/jlb647/scratch/simulation_program/hermes-3_sim/simulation_dir/2024-10_glimmer_working_cases/SNB/CD')
snb_sxd = xh.open('/users/jlb647/scratch/simulation_program/hermes-3_sim/simulation_dir/2024-10_glimmer_working_cases/SNB/SXD')

/users/jlb647/scratch/simulation_program/hermes-3_sim/analysis/hermes-3_venv/lib/python3.11/site-packages/xarray/core/concat.py:527: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))


In [ ]:
def plot_te_ne_nd(ds, title = ''):
    
    ds = ds.isel(t=-1)
    Te = replace_guards(np.ravel(ds['Te']))
    Ne = replace_guards(np.ravel(ds['Ne']))
    Nd = replace_guards(np.ravel(ds['Nd']))
    y = ds['y'][1:-1]

    fig,ax = plt.subplots(1,1, figsize = (12,6), dpi = 500)
    ax.plot(y,Te, label = 'Te')
    ax.set_ylabel('Te (eV)')
    ax.set_xlabel(rf'S$_\parallel$ (m)')
    ax2 = ax.twinx()

    ax2.plot(y,Ne, label = 'Ne', color = 'red')
    ax2.plot(y,Nd, label = 'Nd', color = 'green', linestyle = '--')
    ax2.set_yscale('log')
    ax2.grid(False)
    ax2.set_ylabel(rf'Ne, Nd (m$^-3$)')

    if title != '':
        ax.set_title(title)

    fig.legend()

In [ ]:
coef0 = 1.7597676684e+00		# Manually setting the flux expansion with a profile from the equilibrium files
coef1 = 4.1883560155e-01		# Very hacky but it gets the job done
coef2 = -4.6906367245e+00		# area_expansion normally does a good enough job
coef3 = 7.3174700447e+00
coef4 = -5.7251396733e+00
coef5 = 2.6799740048e+00
coef6 = -7.9457620000e-01
coef7 = 1.5075033673e-01
coef8 = -1.7757965412e-02
coef9 = 1.1834284917e-03
coef10 = -3.4102000406e-05

y = np.linspace(0,2*np.pi,100)


J = 0  + y^0*coef0 + y^1*coef1 + y^2*coef2 + y^3*coef3 + y^4*coef4 + y^5*coef5 + y^6*coef6 + y^7*coef7 + y^8*coef8 + y^9*coef9 + y^10*coef10